In [ ]:
import io
from google.colab import files
import pandas as pd
import folium


uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['flights_final.csv']))

Saving flights_final.csv to flights_final.csv


In [ ]:
import numpy as np
df.head(20)
global datos
datos=df.values.tolist()


In [ ]:
unicos = pd.concat([df['Source Airport Code'], df['Destination Airport Code']], ignore_index=True)
unicos = unicos.unique().tolist()
distancias = []
for i in range(len(datos)):
        lat1 = np.radians(datos[i][4])
        lon1 = np.radians(datos[i][5])
        lat2 = np.radians(datos[i][10])
        lon2 = np.radians(datos[i][11])

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
        c = 2 * np.arcsin(np.sqrt(a))
        radius = 6372.8
        distancia = c * radius
        distancias.append([datos[i][0], datos[i][6], distancia])


In [ ]:
#Se grafican todos los aereopuertos.
m = folium.Map(location=[41.9, -97.3], zoom_start=4, tiles = "cartodbpositron")
for i in unicos:
  ic =folium.Icon(color = "lightgray", icon='plane')
  x = df[df['Source Airport Code'] == i]
  if not x.empty:
    latitude = x.iloc[0]['Source Airport Latitude']
    longitude = x.iloc[0]['Source Airport Longitude']
    tooltip = x.iloc[0]['Source Airport Name']
    pp = f"Código: {i}\nCiudad/País: {x.iloc[0]['Source Airport City']}, {x.iloc[0]['Source Airport Country']}\nLat, lon: {[latitude, longitude]}"
  if x.empty:
    x = df[df['Destination Airport Code'] == i]
    latitude = x.iloc[0]['Destination Airport Latitude']
    longitude = x.iloc[0]['Destination Airport Longitude']
    tooltip = x.iloc[0]['Destination Airport Name']
    pp = f"Código: {i}\nCiudad/País: {x.iloc[0]['Destination Airport City']}, {x.iloc[0]['Destination Airport Country']}\nLat, lon: {[latitude, longitude]}"
   # nombre del marcador
  folium.Marker([latitude, longitude], popup=pp, tooltip=tooltip, icon = ic).add_to(m)
m

FUNCIONES IMPORTANTES

In [ ]:
import math
import networkx as nx
from typing import Any, List, Optional, Tuple, Union, Dict
from pprint import pprint
from queue import Queue
from pprint import pprint
import heapq

def send_cod_by_number(number): # Dado un número retorna el código de aeropuerto correspondiente.
  for i in range(len(unicos)):
    if i==number:
      return unicos[i]
def search(cod): # Buscar info según código
  for i in range(len(datos)):
    if datos[i][0]==cod:
      print('Código: ',datos[i][0])
      print('\nNombre: ',datos[i][1])
      print('\nCiudad: ',datos[i][2])
      print('\nPaís: ',datos[i][3])
      print('\nLatitude: ',datos[i][4])
      print('\nLongitude: ',datos[i][5])
      return
  print("\n  Dicho codigo no esta en la base de datos. \n ")

class WeightedGraph:
    INF = math.inf
    def __init__(self, n: int, directed: bool = False) -> None:
        self.n = n
        self.directed = directed
        self.C = [[self.INF if i != j else 0 for i in range(n)] for j in range(n)]
    def add_edge(self, vi: int, vf: int, weight: int = 1) -> bool:
      if vi is not None and vf is not None:
          if not (0 <= vi < self.n) or not (0 <= vf < self.n):
              return False
          self.C[vi][vf] = weight
          if not self.directed:
              self.C[vf][vi] = weight
          return True
max_graph= WeightedGraph(len(unicos),False)
for distancia in distancias:
  max_graph.add_edge(unicos.index(distancia[0]),unicos.index(distancia[1]),distancia[2])




In [ ]:
import heapq

class Grafo:
    def __init__(self, vertices):
        self.V = vertices
        self.grafo = [[0 for column in range(vertices)] for row in range(vertices)]
    def imprimirsol(self, ddst):

        for distance, vertex in ddst:
            print(f"{unicos[vertex]}   {distance}")
            print(search(unicos[vertex]),"\n\n\n")
    def distmin(self, dist, cam_min):
        min = float('inf')
        min_index = None
        for v in range(self.V):
            if dist[v] < min and not cam_min[v]:
                min = dist[v]
                min_index = v
        return min_index
    def dijkstra_10(self, src):
        dist = [float('inf')] * self.V
        dist[src] = 0
        cam_min = [False] * self.V
        ddst = []

        for _ in range(self.V):
            u = self.distmin(dist, cam_min)
            if u is None:
                break
            cam_min[u] = True
            for v in range(self.V):
                if (self.grafo[u][v] > 0 and
                   not cam_min[v] and
                   dist[v] > dist[u] + self.grafo[u][v]):
                    dist[v] = dist[u] + self.grafo[u][v]

                    ddst.append((dist[v], v))
                    if len(ddst) > 10:
                        ddst.sort(reverse=True)
                        ddst = ddst[:10]

        self.imprimirsol(ddst)


In [ ]:
#Necesarias para hallar caminos mínimos entre dos vertices dados
import math

# distancia mínima con nodos no visitados
def dmin(d, v):
    dminima = math.inf
    vminimo = None
    for node, dist in enumerate(d):

        if dist < dminima and node not in v:
            dminima = dist
            vminimo = node
    return vminimo

# se aplica dijkstra
def dijkstra(adj, v0, v1):
    nnodos = len(adj)
    d = [math.inf] * nnodos
    p = [None] * nnodos
    v = set()
    d[v0] = 0
    while len(v) < nnodos:
        nact = dmin(d, v)
        if nact is None:
            break
        v.add(nact)
        for vec, peso in enumerate(adj[nact]):
            if peso > 0:
                distancia = d[nact] + peso
                if distancia < d[vec]:
                    d[vec] = distancia
                    p[vec] = nact
    camino = []
    nact = v1
    while nact is not None:
        camino.insert(0, nact)
        nact = p[nact]
    return d[v1], camino

In [ ]:
import random
print("Menu")
seguir="s"
x=random.randint(0,len(datos))
while (seguir.lower()=='s'):
  print(" 1. Consultar un aeropuerto. \n 2. Consultar una ruta.")
  opc=int(input("Digite la opcion que quiere escoger: "))
  if opc==1:
      print(" a. ¿Desea ver información sobre algún aeropuerto en específico? \n b. ¿Desea saber cuáles son los 10 destinos más lejanos yendo por las rutas mínimas?")
      a_b=input(' ')
      print("Aquí hay algunos códigos para explorar:")
      print('\n')
      cods = {datos[i][0] for i in range(x,x+20)}
      print(cods)
      print('\n')
      global codigo
      codigo =input("Digite el código del aeropuerto: \n")
      if a_b=='a':
         print("\n")
         search(codigo)
      elif a_b=='b':
        adj = Grafo(len(unicos))
        adj.grafo = max_graph.C
        print("Distancias mínimas mayores desde ",codigo)
        adj.dijkstra_10(unicos.index(codigo))
        print("Para ver el camino en el mapa siga con la siguiente celda")
  if opc == 2:
    print("Por favor, ingrese los dos aeropuertos:")
    print("Aquí hay algunos códigos para explorar:")
    print('\n')
    cods = {datos[i][0] for i in range(x,x+20)}
    print('\n')
    while True:
      v0 = input("Ingrese el primer vértice (v0): ")
      v1 = input("Ingrese el segundo vértice (v1): ")

      if v0 in unicos and v1 in unicos:
          distancia, camino = dijkstra(max_graph.C, unicos.index(v0), unicos.index(v1))
          for i in camino:
              print(unicos[i], end=' -> ')
          print(f"Distancia mínima: {distancia}")
          break  # Salir del bucle while
      else:
          print("Digite códigos válidos, por favor.")
  seguir=input("\n¿Quiere ver otra opción? En ese caso digite S. Si no, escriba cualquier otra letra.")
print('\n')
print(" ''''''''''''''''''''''''''''''''''''''''''''''' ")
print('Gracias por haber usado nuestro menu')
print(" ''''''''''''''''''''''''''''''''''''''''''''''' ")


Menu
 1. Consultar un aeropuerto. 
 2. Consultar una ruta.
Digite la opcion que quiere escoger: 1
 a. ¿Desea ver información sobre algún aeropuerto en específico? 
 b. ¿Desea saber cuáles son los 10 destinos más lejanos yendo por las rutas mínimas?
 a
Aquí hay algunos códigos para explorar:


{'PNH', 'DLM', 'BDA', 'VLC', 'AUA', 'AYT', 'SAW', 'CAG', 'OLB', 'MQT', 'AGP', 'ADB', 'SKG', 'PEK', 'YIW', 'ATL', 'IGU', 'AUH', 'PHL', 'LGW'}


Digite el código del aeropuerto: 
DLM


Código:  DLM

Nombre:  Dalaman International Airport

Ciudad:  Dalaman

País:  Turkey

Latitude:  36.71310043

Longitude:  28.79249954

¿Quiere ver otra opción? En ese caso digite S. Si no, escriba cualquier otra letra.s
 1. Consultar un aeropuerto. 
 2. Consultar una ruta.
Digite la opcion que quiere escoger: 2
Por favor, ingrese los dos aeropuertos:
Aquí hay algunos códigos para explorar:




Ingrese el primer vértice (v0): YIW
Ingrese el segundo vértice (v1): PNH
YIW -> CAN -> PNH -> Distancia mínima: 2534.91802309

In [ ]:
#grafíca el camino mínimo entre los dos nodos dados
m = folium.Map(location=[41.9, -97.3], zoom_start=4, tiles = "cartodbpositron")

latlon = []
for i in camino:
  ic =folium.Icon(color = "gray", icon='plane')
  if i == camino[0]:
    ic = folium.Icon(color="green", prefix='fa',icon='plane')
  if i == camino[-1]:
    ic = folium.Icon(color="red",prefix='fa',icon='plane')
  x = df[df['Source Airport Code'] == unicos[i]]
  if not x.empty:
    latitude = x.iloc[0]['Source Airport Latitude']
    longitude = x.iloc[0]['Source Airport Longitude']
    tooltip = x.iloc[0]['Source Airport Name']
    pp = f"Código: {unicos[i]}\nCiudad/País: {x.iloc[0]['Source Airport City']}, {x.iloc[0]['Source Airport Country']}\nLat, lon: {[latitude, longitude]}"
  if x.empty:
    x = df[df['Destination Airport Code'] == unicos[i]]
    latitude = x.iloc[0]['Destination Airport Latitude']
    longitude = x.iloc[0]['Destination Airport Longitude']
    tooltip = x.iloc[0]['Destination Airport Name']
    pp = f"Código: {unicos[i]}\nCiudad/País: {x.iloc[0]['Destination Airport City']}, {x.iloc[0]['Destination Airport Country']}\nLat, lon: {[latitude, longitude]}"

   # nombre del marcador
  latlon.append([latitude, longitude])
  folium.Marker([latitude, longitude], popup=pp, tooltip=tooltip, icon = ic).add_to(m)

folium.PolyLine(latlon, color='red', weight=1, opacity=1, tooltip = tooltip).add_to(m)
m